# Training with RLlib PPO
[RLlib](https://docs.ray.io/en/latest/rllib/index.html) is a high-performance, distributed
reinforcement learning library. It is preferable to other RL libraries (e.g. Stable Baselines
3) for `bsk_rl` environments because it steps environments copies asynchronously; because 
of the variable step lengths, variable episode step counts, and long episode reset times, 
stepping each environment independently can increase step throughput by 2-5 times.

<div class="alert alert-warning">

**Warning:** RLlib currently has a bug that results in an undesirable timeout which stops
training. Check here to see if it has been resolved: https://github.com/ray-project/ray/pull/45147

</div>


## Define the Environment
A nadir-scanning environment is created, to the one used in [this paper](https://hanspeterschaub.info/Papers/Stephenson2024.pdf). 
The satellite has to collect data while managing the data buffer level and battery level.

First, the satellite class is defined. A custom dynamics model is created that defines
a few additional properties to use in the state.

In [1]:
import numpy as np
from bsk_rl import act, data, obs, sats, scene
from bsk_rl.sim import dyn, fsw

class ScanningDownlinkDynModel(dyn.ContinuousImagingDynModel, dyn.GroundStationDynModel):
    # Define some custom properties to be accessed in the state
    @property
    def instrument_pointing_error(self) -> float:
        r_BN_P_unit = self.r_BN_P/np.linalg.norm(self.r_BN_P) 
        c_hat_P = self.satellite.fsw.c_hat_P
        return np.arccos(np.dot(-r_BN_P_unit, c_hat_P))
    
    @property
    def solar_pointing_error(self) -> float:
        a = self.world.gravFactory.spiceObject.planetStateOutMsgs[
            self.world.sun_index
        ].read().PositionVector
        a_hat_N = a / np.linalg.norm(a)
        nHat_B = self.satellite.sat_args["nHat_B"]
        NB = np.transpose(self.BN)
        nHat_N = NB @ nHat_B
        return np.arccos(np.dot(nHat_N, a_hat_N))

class ScanningSatellite(sats.AccessSatellite):
    observation_spec = [
        obs.SatProperties(
            dict(prop="storage_level_fraction"),
            dict(prop="battery_charge_fraction"),
            dict(prop="wheel_speeds_fraction"),
            dict(prop="instrument_pointing_error", norm=np.pi),
            dict(prop="solar_pointing_error", norm=np.pi)
        ),
        obs.Eclipse(),
        obs.OpportunityProperties(
            dict(prop="opportunity_open", norm=5700),
            dict(prop="opportunity_close", norm=5700),
            type="ground_station",
            n_ahead_observe=1,
        ),
        obs.Time(),
    ]
    action_spec = [
        act.Scan(duration=180.0),
        act.Charge(duration=180.0),
        act.Downlink(duration=60.0),
        act.Desat(duration=60.0),
    ]
    dyn_type = ScanningDownlinkDynModel
    fsw_type = fsw.ContinuousImagingFSWModel

Next, parameters are set. Since this scenario is focused on maintaining acceptable data
and power levels, these are tuned to create a sufficiently interesting mission.

In [2]:
sat = ScanningSatellite(
    "Scanner-1",
    sat_args=dict(
        # Data
        dataStorageCapacity=5000 * 8e6,  # MB to bits
        storageInit=lambda: np.random.uniform(0, 5000 * 8e6),
        instrumentBaudRate=0.5e6,
        transmitterBaudRate=-112e6,
        # Power
        batteryStorageCapacity=400 * 3600,  # Wh to W*s
        storedCharge_Init=lambda: np.random.uniform(400 * 3600 * 0.2, 400 * 3600 * 0.8),
        basePowerDraw=-10.0,
        instrumentPowerDraw=-30.0,
        transmitterPowerDraw=-25.0,
        thrusterPowerDraw=-80.0,
        # Attitude
        imageAttErrorRequirement=0.1,
        imageRateErrorRequirement=0.1,
        disturbance_vector=lambda: np.random.normal(scale=0.0001, size=3),
        maxWheelSpeed=6000.0,  # RPM
        wheelSpeeds=lambda: np.random.uniform(-3000, 3000, 3),
        desatAttitude="nadir",
        nHat_B=np.array([0, 0, -1]),  # Solar panel orientation
    )
)

Finally, the environment arguments are set. Stepping through this environment is 
demonstrated at the bottom of the page.

In [3]:
duration = 2 * 5700.0  # About 2 orbits
env_args = dict(
    satellite=sat,
    scenario=scene.UniformNadirScanning(value_per_second=1/duration),
    rewarder=data.ScanningTimeReward(),
    time_limit=duration,
    failure_penalty=-1.0,
    terminate_on_time_limit=True,
)

## Configure Ray and PPO

The `bsk_rl` package supplies a utility to make logging information at the end of episodes
easier. This is useful to see how an agent's policy is changing over time, using a
monitoring program such as [TensorBoard](https://www.tensorflow.org/tensorboard).

In [4]:
from bsk_rl.utils.rllib import EpisodeDataCallbacks

class CustomDataCallbacks(EpisodeDataCallbacks):
    def pull_env_metrics(self, env):
        reward = env.rewarder.cum_reward
        reward = sum(reward.values()) / len(reward)
        orbits = env.simulator.sim_time / (95 * 60)

        data = dict(
            reward=reward,
            # Are satellites dying, and how and when?
            alive=float(env.satellite.is_alive()),
            rw_status_valid=float(env.satellite.dynamics.rw_speeds_valid()),
            battery_status_valid=float(env.satellite.dynamics.battery_valid()),
            orbits_complete=orbits,
        )
        if orbits > 0:
            data["reward_per_orbit"] = reward / orbits
        if not env.satellite.is_alive():
            data["orbits_complete_partial_only"] = orbits
            
        return  data

Then, PPO (or some other algorithm) can be configured. Of particular importance
are setting `sample_timeout_s` and `metrics_episode_collection_timeout_s` to appropriately
high values for this environment.

In [5]:
from bsk_rl import SatelliteTasking
from bsk_rl.utils.rllib import unpack_config
from ray.rllib.algorithms.ppo import PPOConfig

training_args = dict(
    lr=0.00003,
    gamma=0.999,
    train_batch_size=250,  # In practice, usually a bigger number
    num_sgd_iter=10,
    model=dict(fcnet_hiddens=[512, 512], vf_share_layers=False),
    lambda_=0.95,
    use_kl_loss=False,
    clip_param=0.1,
    grad_clip=0.5,
)

config = (
    PPOConfig()
    .training(**training_args)
    .env_runners(num_env_runners=2, sample_timeout_s=1000.0)
    .environment(
        env=unpack_config(SatelliteTasking),
        env_config=env_args,
    )
    .callbacks(CustomDataCallbacks)
    .reporting(
        metrics_num_episodes_for_smoothing=1,
        metrics_episode_collection_timeout_s=180,
    )
    .checkpointing(export_native_model_files=True)
    .framework(framework="tf2")
    # Uncomment these lines if using the new RLlib API stack
    # .framework(framework="torch")
    # .api_stack(
    #     enable_rl_module_and_learner=True,
    #     enable_env_runner_and_connector_v2=True,
    # )

)

Once the PPO configuration has been set, `ray` can be started and the agent can be
trained.

Training on a reasonably modern machine, we can achieve 5M steps over 32 processors in 6
to 18 hours, depending on specific environment configurations.

In [6]:
import ray
from ray import tune

ray.init(
    ignore_reinit_error=True,
    num_cpus=3,
    object_store_memory=2_000_000_000,  # 2 GB
)

# Run the training
tune.run(
    "PPO",
    config=config.to_dict(),
    stop={"training_iteration": 10},  # Adjust the number of iterations as needed
    checkpoint_freq=10,
    checkpoint_at_end=True
)

# Shutdown Ray
ray.shutdown()

2024-07-15 13:46:23,931	INFO worker.py:1788 -- Started a local Ray instance.


2024-07-15 13:46:24,222	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


/Users/markstephenson/avslab/refactor/.venv_refactor/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/Users/markstephenson/avslab/refactor/.venv_refactor/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/markstephenson/avslab/refactor/.venv_refactor/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


2024-07-15 13:46:24,257	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_cpus_per_worker` has been deprecated. Use `AlgorithmConfig.num_cpus_per_env_runner` instead. This will raise an error in the future!


2024-07-15 13:46:24,257	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_gpus_per_worker` has been deprecated. Use `AlgorithmConfig.num_gpus_per_env_runner` instead. This will raise an error in the future!


...

(PPO pid=77808) 2024-07-15 13:46:35,181	WARNING tf_utils.py:797 -- KL divergence is non-finite, this will likely destabilize your model and the training process. Action(s) in a specific state have near-zero probability. This can happen naturally in deterministic environments where the optimal policy has zero mass for a specific action. To fix this issue, consider setting the coefficient for the KL loss term to zero or increasing policy entropy.


Trial name,agent_timesteps_total,counters,custom_metrics,env_runners,episode_media,fps,info,num_agent_steps_sampled,num_agent_steps_sampled_lifetime,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_lifetime,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_healthy_workers,num_in_flight_async_sample_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,timers
PPO_SatelliteTasking_Unpacked_4ca59_00000,2500,"{'num_env_steps_sampled': 2500, 'num_env_steps_trained': 2500, 'num_agent_steps_sampled': 2500, 'num_agent_steps_trained': 2500}",{},"{'episode_reward_max': 0.6009649122807019, 'episode_reward_min': 0.5163157894736842, 'episode_reward_mean': 0.5635964912280702, 'episode_len_mean': 81.0, 'episode_media': {}, 'episodes_timesteps_total': 324, 'policy_reward_min': {'default_policy': 0.5163157894736842}, 'policy_reward_max': {'default_policy': 0.6009649122807019}, 'policy_reward_mean': {'default_policy': 0.5635964912280702}, 'custom_metrics': {'reward_mean': 0.5635964912280702, 'reward_min': 0.5163157894736842, 'reward_max': 0.6009649122807019, 'alive_mean': 1.0, 'alive_min': 1.0, 'alive_max': 1.0, 'rw_status_valid_mean': 1.0, 'rw_status_valid_min': 1.0, 'rw_status_valid_max': 1.0, 'battery_status_valid_mean': 1.0, 'battery_status_valid_min': 1.0, 'battery_status_valid_max': 1.0, 'orbits_complete_mean': 2.0, 'orbits_complete_min': 2.0, 'orbits_complete_max': 2.0, 'reward_per_orbit_mean': 0.2817982456140351, 'reward_per_orbit_min': 0.2581578947368421, 'reward_per_orbit_max': 0.30048245614035096}, 'hist_stats': {'episode_reward': [0.5163157894736842, 0.5921929824561404, 0.6009649122807019, 0.5449122807017543], 'episode_lengths': [84, 80, 82, 78], 'policy_default_policy_reward': [0.5163157894736842, 0.5921929824561404, 0.6009649122807019, 0.5449122807017543]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 3.12638492416516, 'mean_inference_ms': 0.6783098149166213, 'mean_action_processing_ms': 0.04792242027301011, 'mean_env_wait_ms': 9.79938514703374, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.002938508987426758, 'StateBufferConnector_ms': 0.0023365020751953125, 'ViewRequirementAgentConnector_ms': 0.04649758338928223}, 'num_episodes': 4, 'episode_return_max': 0.6009649122807019, 'episode_return_min': 0.5163157894736842, 'episode_return_mean': 0.5635964912280702, 'episodes_this_iter': 4}",{},123.104,"{'learner': {'default_policy': {'learner_stats': {'cur_kl_coeff': 0.0003906250058207661, 'cur_lr': 2.9999999242136255e-05, 'total_loss': 0.009654259, 'policy_loss': -0.0056497613, 'vf_loss': 0.015303092, 'vf_explained_var': -0.00025595128, 'kl': 0.002373131, 'entropy': 1.1739938, 'entropy_coeff': 0.0}, 'custom_metrics': {}, 'num_agent_steps_trained': 125.0, 'num_grad_updates_lifetime': 190.5, 'diff_num_grad_updates_vs_sampler_policy': 9.5}}, 'num_env_steps_sampled': 2500, 'num_env_steps_trained': 2500, 'num_agent_steps_sampled': 2500, 'num_agent_steps_trained': 2500}",2500,2500,2500,2500,2500,250,123.237,2500,250,123.237,2,0,0,250,"{'cpu_util_percent': 19.466666666666665, 'ram_util_percent': 84.23333333333333}","{'training_iteration_time_ms': 1891.949, 'restore_workers_time_ms': 0.01, 'training_step_time_ms': 1891.919, 'sample_time_ms': 1766.518, 'learn_time_ms': 121.545, 'learn_throughput': 2056.844, 'synch_weights_time_ms': 3.65}"


(PPO pid=77808) WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
(PPO pid=77808) WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2024-07-15 13:46:52,374	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/markstephenson/ray_results/PPO_2024-07-15_13-46-24' in 0.0179s.


(PPO pid=77808) /Users/markstephenson/avslab/refactor/.venv_refactor/lib/python3.10/site-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'function'>, which may lead to improper serialization.
(PPO pid=77808)   warnings.warn(
(PPO pid=77808) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/markstephenson/ray_results/PPO_2024-07-15_13-46-24/PPO_SatelliteTasking_Unpacked_4ca59_00000_0_2024-07-15_13-46-24/checkpoint_000000)


2024-07-15 13:46:52,877	INFO tune.py:1041 -- Total run time: 28.65 seconds (28.10 seconds for the tuning loop).


## Loading the Policy Network

The policy network can be found in the `model` subdirectory of the checkpoint output.

## Stepping Through the Environment

The environment is stepped through with random actions to give a sense of how it acts.

In [7]:
env = SatelliteTasking(**env_args, log_level="INFO")
env.reset()
terminated = False
while not terminated:
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

2024-07-15 13:46:54,392 gym                            INFO       Resetting environment with seed=1462109616


2024-07-15 13:46:54,486 sats.satellite.Scanner-1       INFO       <0.00> Scanner-1: Finding opportunity windows from 0.00 to 11400.00 seconds


2024-07-15 13:46:54,512 gym                            INFO       <0.00> Satellites requiring retasking: ['Scanner-1_4715111056']


2024-07-15 13:46:54,512 gym                            INFO       <0.00> Environment reset


2024-07-15 13:46:54,513 gym                            INFO       <0.00> === STARTING STEP ===


...

2024-07-15 13:46:55,475 sim.simulator                  INFO       <11340.00> Running simulation at most to 11400.00 seconds


2024-07-15 13:46:55,479 data.base                      INFO       <11400.00> Data reward: {'Scanner-1_4715111056': 0.0}


2024-07-15 13:46:55,479 gym                            INFO       <11400.00> Step reward: 0.0


2024-07-15 13:46:55,480 gym                            INFO       <11400.00> Episode terminated: True


2024-07-15 13:46:55,480 gym                            INFO       <11400.00> Episode truncated: True
